# Sheet 08

In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer


## Exercise 1: Multi-label classification

In [32]:
df = pd.read_csv('../data/JanataHack.csv', index_col='ID')
df

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
ID,,,,,,,,
1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...
20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0


In [36]:
df.rename(columns={"TITLE": "title", "ABSTRACT" : "abstract"}, inplace=True)

First we need to do some cleaning.

In [39]:
# first we need to do some cleaning
df_abstracts = df['abstract']
df_abstracts

ID
1          Predictive models allow subject-specific inf...
2          Rotation invariance and translation invarian...
3          We introduce and develop the notion of spher...
4          The stochastic Landau--Lifshitz--Gilbert (LL...
5          Fourier-transform infra-red (FTIR) spectra o...
                               ...                        
20968      Machine learning is finding increasingly bro...
20969      Polycrystalline diamond coatings have been g...
20970      We present a new approach for identifying si...
20971      The sum of Log-normal variates is encountere...
20972      Recently, optional stopping has been a subje...
Name: abstract, Length: 20972, dtype: object

In [40]:
# extract features from the text with 'bag of words'
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df_abstracts)

In [16]:

# labels for the categories
labels = ["Computer Science", "Physics", "Mathematics", "Statistics", "Quantitative Biology", "Quantitative Finance"]